Deep Learning
=============

Regularization
------------

Previously we trained a logistic regression and a neural network model. The goal here is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 28, 28), (200000,))
('Validation set', (10000, 28, 28), (10000,))
('Test set', (10000, 28, 28), (10000,))


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (200000, 784), (200000, 10))
('Validation set', (10000, 784), (10000, 10))
('Test set', (10000, 784), (10000, 10))


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
L2 Regularization
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

###Logistic Regression:

In [5]:
batch_size = 512

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # See modified loss function with REGULARIZATION TERM   
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)+ 1e-3*tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 37.772808
Minibatch accuracy: 6.1%
Validation accuracy: 6.9%
Minibatch loss at step 500: 13.906009
Minibatch accuracy: 68.6%
Validation accuracy: 67.6%
Minibatch loss at step 1000: 10.148291
Minibatch accuracy: 75.8%
Validation accuracy: 72.6%
Minibatch loss at step 1500: 7.796410
Minibatch accuracy: 77.5%
Validation accuracy: 74.4%
Minibatch loss at step 2000: 5.982008
Minibatch accuracy: 79.9%
Validation accuracy: 75.6%
Minibatch loss at step 2500: 4.752683
Minibatch accuracy: 76.8%
Validation accuracy: 76.9%
Minibatch loss at step 3000: 3.782213
Minibatch accuracy: 77.7%
Validation accuracy: 78.0%
Test accuracy: 84.1%


### One Hidden ReLU Layer NN

In [7]:
#batch_size = 8 # for problem 2; also turn regularization constant to 0
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))  
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu_o = tf.nn.relu(logits1)
  logits2 = tf.matmul(relu_o,weights2)+biases2  
  regularizer = tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))+5e-4*regularizer)

  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(
                tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2)

In [8]:
num_steps = 3001
#train_dataset1 = train_dataset[1:500,:] # Comment to avoid overfitting. Also turn of regularization above
#train_labels1 = train_labels[1:500,:]   # Comment to avoid overfitting.

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 494.880646
Minibatch accuracy: 7.8%
Validation accuracy: 27.5%
Minibatch loss at step 500: 165.171448
Minibatch accuracy: 82.0%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 154.134750
Minibatch accuracy: 85.2%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 144.865021
Minibatch accuracy: 81.2%
Validation accuracy: 83.7%
Minibatch loss at step 2000: 135.017075
Minibatch accuracy: 79.7%
Validation accuracy: 83.1%
Minibatch loss at step 2500: 123.932602
Minibatch accuracy: 89.1%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 117.566750
Minibatch accuracy: 88.3%
Validation accuracy: 84.9%
Test accuracy: 90.2%


---
Overfitting
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

We get 100% accuracy on training data; however, both validation and test accuracy remains low.

---

---
Dropout to Prevent Overfitting
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
#batch_size = 8 # for problem 2; also turn regularization constant to 0
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))  
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu_o = tf.nn.relu(logits1)
  logits2 = tf.matmul(tf.nn.dropout(relu_o,0.5),weights2)+biases2  
  regularizer = tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))+.0005*regularizer)

  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.1
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100000, 0.96, staircase=True)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(tf.matmul(
    tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(
                tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2)

In [10]:
num_steps = 3001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 654.644409
Minibatch accuracy: 8.6%
Validation accuracy: 28.9%
Minibatch loss at step 500: 175.224960
Minibatch accuracy: 78.1%
Validation accuracy: 83.7%
Minibatch loss at step 1000: 164.034470
Minibatch accuracy: 73.4%
Validation accuracy: 84.0%
Minibatch loss at step 1500: 144.562119
Minibatch accuracy: 72.7%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 135.114716
Minibatch accuracy: 76.6%
Validation accuracy: 82.7%
Minibatch loss at step 2500: 121.736450
Minibatch accuracy: 87.5%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 117.341919
Minibatch accuracy: 75.8%
Validation accuracy: 81.9%
Test accuracy: 87.7%


---
Improve Performance
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [25]:
#batch_size = 8 # for problem 2; also turn regularization constant to 0
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024],stddev=np.sqrt(2./784.)))
  biases1 = tf.Variable(tf.zeros([1024]))
  
  weights2 = tf.Variable(
    tf.truncated_normal([1024, 1024],stddev = np.sqrt(2./1024.)))
  biases2 = tf.Variable(tf.zeros([1024]))

  weights3 = tf.Variable(
    tf.truncated_normal([1024, 300],stddev = np.sqrt(2./1024.)))
  biases3 = tf.Variable(tf.zeros([300]))
  
  weights4 = tf.Variable(
    tf.truncated_normal([300, num_labels],stddev =np.sqrt(2/300.)))
  biases4 = tf.Variable(tf.zeros([num_labels]))
  
  
  # Training computation.
  l1_input = tf.matmul(tf_train_dataset, weights1) + biases1
  l1_output = tf.nn.dropout(tf.nn.relu(l1_input),0.5)
  l2_input = tf.matmul(l1_output,weights2)+biases2
  l2_output = tf.nn.dropout(tf.nn.relu(l2_input),0.5)
  l3_input = tf.matmul(l2_output,weights3)+biases3
  l3_output = tf.nn.dropout(tf.nn.relu(l3_input),0.75)     
  logits = tf.matmul(l3_output,weights4)+biases4

  train_prediction = tf.nn.softmax(logits)
         
  regularizer = tf.nn.l2_loss(weights1)+tf.nn.l2_loss(weights2)+tf.nn.l2_loss(weights3)+tf.nn.l2_loss(weights4)
  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+.001*regularizer)

  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.25
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           1000, 0.95, staircase=True)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)  
  # The following steps are without dropouts
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2),weights3)+biases3),weights4)+biases4)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2),weights3)+biases3),weights4)+biases4)

In [26]:
num_steps = 20001
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.501478
Minibatch accuracy: 11.7%
Validation accuracy: 23.9%
Minibatch loss at step 2000: 1.239352
Minibatch accuracy: 85.2%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.739006
Minibatch accuracy: 88.3%
Validation accuracy: 88.0%
Minibatch loss at step 6000: 0.783015
Minibatch accuracy: 84.4%
Validation accuracy: 89.2%
Minibatch loss at step 8000: 0.544449
Minibatch accuracy: 90.6%
Validation accuracy: 89.1%
Minibatch loss at step 10000: 0.586595
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 12000: 0.672157
Minibatch accuracy: 82.0%
Validation accuracy: 89.4%
Minibatch loss at step 14000: 0.360126
Minibatch accuracy: 93.0%
Validation accuracy: 89.6%
Minibatch loss at step 16000: 0.480915
Minibatch accuracy: 89.1%
Validation accuracy: 89.9%
Minibatch loss at step 18000: 0.401778
Minibatch accuracy: 92.2%
Validation accuracy: 89.6%
Minibatch loss at step 20000: 0.337281
Minibatch accuracy: 95.3%
Validation 